In [2]:
import os
os.chdir('../')
os.getcwd()

'/Users/dennis/repos/study/mt/mt_rdf'

In [3]:
import pandas as pd
from tqdm import tqdm
from functools import reduce

In [4]:
virt_orig_df = pd.read_csv('results/wdbench/ppaths/virt/results_ppaths_original.csv')
virt_orig_df_forced = pd.read_csv('results/wdbench/ppaths/virt/results_ppaths_original_forced.csv')
virt_orig_df_opt = pd.read_csv('results/wdbench/ppaths/virt/results_ppaths_opt_virt.csv')
virt_orig_df.drop('Unnamed: 0', axis=1, inplace=True)
virt_orig_df_forced.drop('Unnamed: 0', axis=1, inplace=True)
virt_orig_df_opt.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
virt_merged_df = pd.merge(virt_orig_df, virt_orig_df_forced, how='left', left_on=['query_id', 'exec_n'], right_on=['query_id', 'exec_n'], suffixes=('', '_forced'))
virt_merged_df = pd.merge(virt_merged_df, virt_orig_df_opt, how='left', left_on=['query_id', 'exec_n'], right_on=['query_id', 'exec_n'], suffixes=('', '_opt'))

In [6]:
virt_merged_df['exec_time'].sum() / 4

1719.3866250514984

In [7]:
virt_merged_df

,query_id,exec_n,exec_time,results,exec_time_forced,results_forced,exec_time_opt,results_opt
0,P365,0,0.034540,NaN,50.981081,NaN,0.037571,NaN
1,P365,1,0.005320,NaN,50.173872,NaN,0.008669,NaN
2,P365,2,0.004696,NaN,50.759371,NaN,0.006482,NaN
3,P365,3,0.003977,NaN,50.263360,NaN,0.004403,NaN
4,P376,0,0.030846,NaN,49.986881,NaN,0.006958,NaN
...,...,...,...,...,...,...,...,...
851,P279,3,0.270709,NaN,432.549118,NaN,NaN,NaN
852,P263,0,0.540939,NaN,5.716760,NaN,NaN,NaN
853,P263,1,0.514997,NaN,5.554682,NaN,NaN,NaN
854,P263,2,0.518899,NaN,5.505023,NaN,NaN,NaN


In [8]:
virt_merged_df[(virt_merged_df['exec_time'] >= virt_merged_df['exec_time_forced']) & virt_merged_df['exec_time_forced'] != 0]

,query_id,exec_n,exec_time,results,exec_time_forced,results_forced,exec_time_opt,results_opt
8,P230,0,295.505047,NaN,42.569398,NaN,NaN,NaN
9,P230,1,246.681965,NaN,43.057276,NaN,NaN,NaN
10,P230,2,246.280662,NaN,43.482172,NaN,NaN,NaN
11,P230,3,247.935213,NaN,44.602695,NaN,NaN,NaN
40,P355,0,3.443652,NaN,1.117381,NaN,0.004738,NaN
...,...,...,...,...,...,...,...,...
759,P275,3,60.404140,NaN,6.054290,NaN,NaN,NaN
784,P285,0,4.393668,NaN,0.192304,NaN,0.021630,NaN
785,P285,1,4.353490,NaN,0.167747,NaN,0.016577,NaN
786,P285,2,4.369842,NaN,0.170515,NaN,0.015898,NaN


In [9]:
virt_opt_not_opt = virt_merged_df[['query_id',  'exec_time', 'exec_time_forced']]

In [10]:
virt_opt_not_opt.loc[virt_opt_not_opt['exec_time_forced'] == 0, 'exec_time_forced'] = 900

In [11]:
virt_opt_not_opt[virt_opt_not_opt['exec_time_forced'] == 900]

,query_id,exec_time,exec_time_forced
32,P357,34.484950,900.0
33,P357,30.141387,900.0
34,P357,30.688312,900.0
35,P357,30.222399,900.0
44,P385,1.028157,900.0
...,...,...,...
839,P315,0.070925,900.0
844,P283,2.621537,900.0
845,P283,2.607222,900.0
846,P283,2.565370,900.0


In [12]:
virt_opt_not_opt.groupby('query_id').apply(lambda x: x['exec_time'].mean() >= x['exec_time_forced'].mean())

query_id
P180    False
P181     True
P182    False
P183    False
P184    False
        ...  
P399    False
P400    False
P401    False
P402    False
P403    False
Length: 214, dtype: bool

In [13]:
dataset = virt_opt_not_opt.groupby('query_id').apply(lambda x: x['exec_time'].mean() >= x['exec_time_forced'].mean())

In [20]:
ds = dataset.to_frame().reset_index().rename(columns={0: 'label'})

In [22]:
ds['label'] = ds['label'].astype(int)

In [23]:
ds

,query_id,label
0,P180,0
1,P181,1
2,P182,0
3,P183,0
4,P184,0
...,...,...
209,P399,0
210,P400,0
211,P401,0
212,P402,0


In [27]:

ds_dict = {'query_id': [], 'query': [], 'label': []}
for index, row in tqdm(ds.iterrows()):
    filename = f'{row["query_id"]}.sparql'

    with open(os.path.join('data/raw/wdbench/ppaths/original', filename), 'r') as file:
        query = file.read()
        ds_dict['query'].append(query)
        ds_dict['query_id'].append(row['query_id'])
        ds_dict['label'].append(row['label'])

final_ds = pd.DataFrame(ds_dict)

214it [00:00, 4816.80it/s]


In [28]:
final_ds

,query_id,query,label
0,P180,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0
1,P181,SELECT * WHERE {\n\t<http://www.wikidata.org/e...,1
2,P182,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0
3,P183,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0
4,P184,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0
...,...,...,...
209,P399,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0
210,P400,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0
211,P401,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0
212,P402,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0


In [29]:
final_ds['query_len'] = final_ds.apply(lambda x: len(x['query']), axis=1)

In [30]:
final_ds['query_len'].mean()

247.07009345794393

In [31]:
final_ds['query_len'].max()

684

In [32]:
final_ds[final_ds['query_len'] > 512].loc[80, 'query']

'SELECT * WHERE {\n\t?x1 <http://www.wikidata.org/prop/direct/P105> ?x2 .\n\t?x3 <http://www.wikidata.org/prop/direct/P105> ?x4 .\n\t?x1 <http://www.wikidata.org/prop/direct/P171> ?x3 .\n\t?x1 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q16521> .\n\t?x3 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q16521> .\n\t?x2 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q427626> .\n\t?x4 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q427626> .\n\t?x4 <http://www.wikidata.org/prop/direct/P361> ?x2 .\n\t?x1 <http://www.wikidata.org/prop/direct/P171>+ <http://www.wikidata.org/entity/Q130942> .\n}'

In [33]:
final_ds

,query_id,query,label,query_len
0,P180,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,196
1,P181,SELECT * WHERE {\n\t<http://www.wikidata.org/e...,1,223
2,P182,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,308
3,P183,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,166
4,P184,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,164
...,...,...,...,...
209,P399,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,456
210,P400,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,219
211,P401,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,163
212,P402,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,164


In [34]:
final_ds[['query_id', 'query', 'label']].to_csv('data/raw/ppaths_join.csv', index=False)